In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install  hf_transfer -q
!pip install flash-attn -q

In [3]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [4]:
import transformers
import torch
from transformers import AutoModelForCausalLM
model_id = "microsoft/Phi-3-mini-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir='',
    device_map='auto',
    torch_dtype=torch.bfloat16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir='')

In [6]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    target_modules=['mlp.gate_up_proj', 'mlp.down_proj'],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=['lm_head', 'embed_tokens'],
    r=128,
    lora_alpha=32,
    use_rslora=True,
)

In [7]:
from peft import get_peft_model
model=get_peft_model(model,lora_config)

model.print_trainable_parameters()

trainable params: 159,186,944 || all params: 1,259,235,328 || trainable%: 12.6416


In [8]:
!pip install git+https://github.com/huggingface/trl.git -q -U

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [9]:
from datasets import load_dataset

dataset_name = "Trelis/openassistant-llama-style"

train_dataset = load_dataset(dataset_name, split="train[:100]")
eval_dataset = load_dataset(dataset_name, split="test[:10]")

print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['text'],
    num_rows: 100
})
Dataset({
    features: ['text'],
    num_rows: 10
})


In [11]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=SFTConfig(
        max_steps=5,
        max_seq_length=2048,
        dataset_text_field="text",
        num_train_epochs=1,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=4,
        do_eval=True,
        lr_scheduler_type="constant",
        learning_rate=1e-4,
        eval_strategy="steps",
        eval_steps=0.2,
        warmup_ratio=0.05,
        output_dir='./',
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": True},
        logging_steps=1,
    )

)

trainer.train()

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss
1,1.590900,1.484849
2,1.589700,1.461789
3,1.585700,1.445649
4,1.528100,1.436540
5,1.195800,1.426608


TrainOutput(global_step=5, training_loss=1.498034381866455, metrics={'train_runtime': 493.4889, 'train_samples_per_second': 0.324, 'train_steps_per_second': 0.01, 'total_flos': 961195155456000.0, 'train_loss': 1.498034381866455, 'epoch': 1.5384615384615383})

In [12]:
trainer.model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-model')


('./fine-tuned-model/tokenizer_config.json',
 './fine-tuned-model/special_tokens_map.json',
 './fine-tuned-model/tokenizer.model',
 './fine-tuned-model/added_tokens.json',
 './fine-tuned-model/tokenizer.json')

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
fine_tuned_model = AutoModelForCausalLM.from_pretrained('./fine-tuned-model', torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained('./fine-tuned-model')


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [19]:
import torch
fine_tuned_model.eval()
fine_tuned_model.to("cuda")

In [27]:
prompt = "ai is a"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = fine_tuned_model.generate(
    inputs.input_ids,
    max_length=1000,
    num_return_sequences=1,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7
)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

ai is a powerful language that can be used to create natural-sounding speech. It has many features that make it useful for creating speech-like applications, such as:

1. Machine Learning: AI-powered models can be trained on large datasets to learn the patterns and features of natural speech. This enables the creation of speech-like applications that can understand and mimic human speech.

2. Speech Synthesis: AI can be used to create speech that is similar to human speech. This can be used to create voice-based interfaces for applications that require natural-sounding speech, such as chatbots or voice-activated assistants.

3. Natural Language Processing: AI can be used to process and understand natural language. This can be used to create applications that can understand and process natural-sounding speech, such as speech-to-text and speech-to-speech translation.

4. Audio Processing: AI can be used to process and analyze audio data. This can be used to create applications that can u

In [28]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


In [31]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [32]:
import torch
import wandb

In [34]:
wandb.init(project="text-generation", config={
    "max_length": 1000,
    "num_return_sequences": 1,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "temperature": 0.7,
})

In [39]:
from transformers import AutoModelForCausalLM, AutoTokenizer
fine_tuned_model = AutoModelForCausalLM.from_pretrained('./fine-tuned-model', torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained('./fine-tuned-model')


In [57]:
import torch
fine_tuned_model.eval()
fine_tuned_model.to("cuda")
prompt = "science is a "
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = fine_tuned_model.generate(
    inputs.input_ids,
    max_length=1000,
    num_return_sequences=1,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7
)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

science is a 400-page tome, and it's a massive work. I could only really scratch the surface of some of the topics and chapters. The book is divided into 13 main chapters, each of which is subdivided into smaller sections. There are also a number of appendices, tables, and figures.

The book is structured in a way that allows the reader to move easily between different sections and topics. Each chapter is introduced with a brief overview of the topic, and then the reader is taken into more in-depth analysis of the subject matter. The book is also arranged in a way that allows the reader to follow the development of the science from its early beginnings to the present day.

The book is written in a style that is both accessible and informative. The authors use a range of language and terminology that is easy for the reader to understand, and they provide a good deal of information and examples to support their arguments.

Overall, the book is a valuable resource for anyone interested in

In [58]:
!wandb login

wandb: Currently logged in as: vickys9715. Use `wandb login --relogin` to force relogin


In [59]:
wandb.init(project="text-generation", config={
    "max_length": 1000,
    "num_return_sequences": 1,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "temperature": 0.7,
})

In [60]:
wandb.log({"generated_text": wandb.Html(generated_text)})

wandb.finish()